D'abord : se logguer à un compte gmail \
login : eleveenmi@gmail.com  (i=1,2,3,..) \
mwd : pwdeleveEnmi (i=1,2,3,..)

# I) Installations et imports

In [ ]:
# Google sheet
!pip install --upgrade gspread gspread_dataframe

In [23]:
# Imports
from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive

import os
import re
import random
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets
from base64 import b64encode
from datetime import datetime

import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe

def text2labels(text):
    # Normalize spaces and underscores
    cleaned = re.sub(r'[\s_]+', ' ', text.strip())

    # Match tokens like C45, h1200, etc.
    matches = re.findall(r'\b([cnh])(\d+)\b', cleaned, flags=re.IGNORECASE)

    if not matches:
        return "SVP repecter le format: Cx Ny Hz avec X,Y,Z des entiers positifs"

    label_dict = {}
    key_map = {'c': 'Classe', 'n': 'Nebul', 'h': 'Hauteur'}

    for key, val in matches:
        val = int(val)
        if val <= 0:
            return "SVP repecter le format: Cx Ny Hz avec X,Y,Z des entiers positifs"
        if key.lower() == 'n' and val > 8:
            return "La nébulosité (N) doit être un entier entre 1 et 8"
        label_dict[key_map[key.lower()]] = val

    return label_dict


def send_labels(dict_labels, filename_to_labels):
    for filename, labels in filename_to_labels.items():
        if isinstance(labels, str):
            return labels  # Return the error immediately
        if filename not in dict_labels:
            return f"l'image n'est pas dans le dictionnaire"
        dict_labels[filename].update(labels)
    return dict_labels


def build_initial_dict_from_images(root_folder='images_to_label'):
    image_data = {}
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.jpg'):
                image_data[filename] = {'Classe': None, 'Nebul': None, 'Hauteur': None}
    return image_data

def labels2color(labels):
        x = labels.get('Classe')
        y = labels.get('Nebul')
        z = labels.get('Hauteur')

        if x is None and y is None and z is None:
            color = 'gray'
        elif x is None:
            color = 'gray'
        elif x == 2:
            color = 'red'
        elif x in {1, 3, 73}:
            color = 'blue'
        elif x == 0:
            color = 'black'
        elif ((x >= 10) and (x <= 29) or x == 80) and x != 23:
            color = 'green'
        elif x in {40, 50}:
            color = 'lightgreen'
        elif 70 <= x <= 72:
            color = '#FAFA33' #lemon
        elif x in {60, 61, 82}:
            color = 'orange'
        elif x == 81:
            color = 'purple'
        elif x in {23, 41, 42, 43, 44}:
            color = 'cyan'
        elif x == 90:
            color = 'pink'
        else:
            color = 'brown'
        return color

def assign_colors(data_dict):
    color_map = {}
    for filename, labels in data_dict.items():
        timestamp = filename.split('_')[1][:-4]
        color_map[timestamp] = labels2color(labels)
    return color_map



# Define the label interpretation function
def get_interpreted_labels(dict_labels, key, correspondance):
    if key is None or key not in dict_labels:
        return "no label"

    labels = dict_labels[key]  # Expected format: {'C': X, 'N': Y, 'H': Z}
    X = labels['Classe']
    return f"{correspondance[int(X)]} (classe {X})" if X in correspondance else f"Pas de labels"



# Placeholder: you must define this earlier
def send_label(text):
    return {"label": text}

# === Functions ===

def load_image_html(image_path, highlight=False):
    with open(image_path, 'rb') as f:
        encoded = b64encode(f.read()).decode()
    ext = os.path.splitext(image_path)[1][1:]
    border_style = "border: 5px solid limegreen;" if highlight else "border: none;"
    return f"""
    <style>
    .hover-container {{
      position: relative;
      width: 800px;
      height: auto;
    }}
    .hover-container img {{
      width: 100%;
      height: auto;
      display: block;
      {border_style}
    }}
    .hover-button {{
      position: absolute;
      background: rgba(255,255,255,0.5);
      border: none;
      font-size: 2em;
      padding: 10px;
      display: none;
      cursor: pointer;
      z-index: 10;
    }}
    .hover-container:hover .hover-button {{
      display: block;
    }}
    #left-btn  {{ left: 0; top: 50%; transform: translateY(-50%); }}
    #right-btn {{ right: 0; top: 50%; transform: translateY(-50%); }}
    #up-btn    {{ top: 0; left: 50%; transform: translateX(-50%); }}
    #down-btn  {{ bottom: 0; left: 50%; transform: translateX(-50%); }}
    </style>

    <div class="hover-container">
        <button id="left-btn" class="hover-button" onclick="google.colab.kernel.invokeFunction('notebook.on_left', [], {{}})">‹</button>
        <button id="right-btn" class="hover-button" onclick="google.colab.kernel.invokeFunction('notebook.on_right', [], {{}})">›</button>
        <button id="up-btn" class="hover-button" onclick="google.colab.kernel.invokeFunction('notebook.on_up', [], {{}})">˄</button>
        <button id="down-btn" class="hover-button" onclick="google.colab.kernel.invokeFunction('notebook.on_down', [], {{}})">˅</button>
        <img src="data:image/{ext};base64,{encoded}" />
    </div>
    """

def on_label_submit(change):
    text = change.value.strip()
    if not text:
        return
    with result_output:
        clear_output()
        try:
            subfolder = subfolders[current_subfolder_index]
            images = sorted([
                f for f in os.listdir(subfolder)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ])
            image_name = images[starting_image_idx] if len(images) > starting_image_idx else None
            if image_name:
                # get labels
                labels = text2labels(text)
                if type(labels) == str:
                  print(labels)
                else :
                  # change button color
                  new_button_color = labels2color(labels)
                  folder_buttons[current_subfolder_index].style.button_color = new_button_color

                  # change button text
                  filename_to_labels = {image_name: labels}  # You defined this function
                  label_text = get_interpreted_labels(filename_to_labels, image_name, correspondance)
                  ts_raw = image_name.split('_')[1][:-4]
                  dt = datetime.strptime(ts_raw, "%Y%m%d%H%M")
                  readable_dt = dt.strftime("%d/%m/%Y %H:%M")
                  full_label = f"{readable_dt}\n{label_text}"
                  folder_buttons[current_subfolder_index].description = full_label

                  # update the dict and the ggogle sheet
                  result = send_labels(dict_labels, filename_to_labels)
                  print(f'annotation {label_text} prise en compte')
                  # update the google sheet
                  update_worksheet_with_dict(worksheet, filename_to_labels)
            else:
                print("No kth image to label.")
        except Exception as e:
            print("Error:", e)
    change.value = ''

def update_image_display():
    subfolder = subfolders[current_subfolder_index]
    images = sorted([
        os.path.join(subfolder, f) for f in os.listdir(subfolder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ])

    global current_image_index
    if not images:
        image_html.value = "<b>No image</b>"
        meta_label.value = f"[{os.path.basename(subfolder)}] — No images"
        meta_label2.value = f"[{os.path.basename(subfolder)}] — No metadata"
        meta_label3.value = f"[{os.path.basename(subfolder)}] — No value"
        status_label.value = ""
        label_input.layout.display = 'none'
        return

    current_image_index %= len(images)
    image_path = images[current_image_index]
    image_html.value = load_image_html(image_path, highlight=(current_image_index == starting_image_idx))

    filename = os.path.basename(image_path)
    direction = filename.split('_')[0] if '_' in filename else '?'
    label_text = get_interpreted_labels(dict_labels, filename, correspondance)
    folder_name = os.path.basename(subfolder)
    parts = folder_name.split('_')
    meta = ", ".join([p for p in parts if '=' in p])
    meta_label.value = f"Direction: {direction}"
    meta_label2.value = f"Metadata: {meta}"
    meta_label3.value = f"Labels: {label_text}"
    status_label.value = f"[{filename} — Image {current_image_index + 1}/{len(images)}]"

    if current_image_index == starting_image_idx:
        label_input.layout.display = 'block'
    else:
        label_input.layout.display = 'none'

    with result_output:
        clear_output()

    display(Javascript('''
    if (!window.simplifiedKeysInitialized) {
      document.addEventListener('keydown', function(event) {
        const tag = document.activeElement.tagName.toLowerCase();
        const isTyping = tag === 'input' || tag === 'textarea';
        if (isTyping) return;
        if (event.key === 'ArrowLeft') {
          google.colab.kernel.invokeFunction('notebook.on_left', [], {});
        } else if (event.key === 'ArrowRight') {
          google.colab.kernel.invokeFunction('notebook.on_right', [], {});
        } else if (event.key === 'a' || event.key === 'A') {
          google.colab.kernel.invokeFunction('notebook.on_up', [], {});
        } else if (event.key === 'z' || event.key === 'Z') {
          google.colab.kernel.invokeFunction('notebook.on_down', [], {});
        }
      });
      window.simplifiedKeysInitialized = true;
    }
    '''))

def show_image_view(index):
    global current_subfolder_index, current_image_index
    current_subfolder_index = index
    current_image_index = starting_image_idx
    update_image_display()
    layout_box.children = [viewer_box]

def return_to_buttons(_=None):
    layout_box.children = [button_box]

# === NAVIGATION CALLBACKS ===
def on_left(b=None):
    global current_image_index
    current_image_index -= 1
    update_image_display()

def on_right(b=None):
    global current_image_index
    current_image_index += 1
    update_image_display()

def on_up(b=None):
    global current_subfolder_index, current_image_index
    current_subfolder_index = (current_subfolder_index - 1) % len(subfolders)
    current_image_index = starting_image_idx
    update_image_display()

def on_down(b=None):
    global current_subfolder_index, current_image_index
    current_subfolder_index = (current_subfolder_index + 1) % len(subfolders)
    current_image_index = starting_image_idx
    update_image_display()


# Manip the google sheet

def update_worksheet(worksheet, data_dict, return_dict_labels=False):
    # Step 1: Load sheet into DataFrame (or create empty one if necessary)
    df_sheet = get_as_dataframe(worksheet, dtype=object, header=0).dropna(how='all')

    if df_sheet.empty or all(c is None or str(c).strip() == '' for c in df_sheet.columns):
        df_sheet = pd.DataFrame(columns=['Image'])

    # Step 2: Create DataFrame from dict
    df_new = pd.DataFrame.from_dict(data_dict, orient='index')
    df_new.index.name = 'Image'
    df_new.reset_index(inplace=True)

    # Step 3: Ensure both DataFrames have the same columns
    all_columns = sorted(set(df_sheet.columns).union(df_new.columns))
    df_sheet = df_sheet.set_index('Image').reindex(columns=[col for col in all_columns if col != 'Image'])
    df_new = df_new.set_index('Image').reindex(columns=[col for col in all_columns if col != 'Image'])

    # Step 4: Insert or update rows from df_new into df_sheet
    df_sheet.update(df_new)              # update existing rows
    df_combined = df_sheet.combine_first(df_new)  # add new rows
    df_combined.reset_index(inplace=True)

    # Step 5: Reorder columns alphabetically (with 'Image' always first)
    ordered_columns = ['Image'] + sorted(c for c in df_combined.columns if c != 'Image')
    df_combined = df_combined[ordered_columns]

    # Step 6: Write back to sheet
    worksheet.clear()
    set_with_dataframe(worksheet, df_combined)
    if return_dict_labels:
      return


def complete_dict_from_worksheet(worksheet, data_dict):
    # Step 1: Load worksheet as DataFrame
    df_sheet = get_as_dataframe(worksheet, dtype=object, header=0).dropna(how='all')
    if df_sheet.empty or 'Image' not in df_sheet.columns:
        return data_dict  # nothing to complete

    df_sheet.set_index('Image', inplace=True)

    # Step 2: For each image in sheet, insert or update in data_dict
    for image_name, row in df_sheet.iterrows():
        row_dict = row.dropna().to_dict()  # skip NaN cells
        if image_name not in data_dict:
            data_dict[image_name] = row_dict
        else:
            for key, val in row_dict.items():
                if data_dict[image_name].get(key) is None:
                    data_dict[image_name][key] = val

    return data_dict

In [5]:
# Define the class ID to class name mapping
correspondance = {
    0: "nsp",
    1: "ciel vide",
    2: "Cb (même loin)",
    3: "ciel quasi vide",
    10: "brouillard et ciel",
    11: "brouillard/brume",
    12: "St fra",
    13: "St neb",
    20: "Cu fra/hu",
    21: "Cu me/con",
    22: "Cu con/Cb (sous)",
    23: "Cu ra",
    30: "Sc",
    40: "Ac str",
    41: "Ac flo",
    42: "Ac len",
    43: "Ac cas",
    44: "Ac/Sc vol",
    50: "As",
    60: "Ns pluie",
    61: "Pannus",
    70: "Ci ou Cs fi",
    71: "Cs neb",
    72: "Cc (et autres Ci/Cs)",
    73: "Ct",
    80: "Cu/Sc sous",
    81: "Nuage turbulent",
    82: "Ns ou St neb op",
    90: "mixte"
}

group2direction_and_starthour = {
    0: ('S', '08H00'), 1: ('S', '10H00'),  # already done
    2: ('S', '08H20'), 3: ('S', '08H40'), 4: ('S', '09H00'), 5: ('S', '09H20'), 6: ('S', '09H40'),  # new times
    7: ('N', '09H00'), 8: ('E', '09H00'), 9: ('O', '09H00'),  # different targets
    10: ('S', '09H10'), 11: ('S', '09H04'), 12: ('S', '09H02'),  # more precise times
    13: ('S', '09H00')  # for Louis
}

group2sheets = {13 : 'https://docs.google.com/spreadsheets/d/1KT4RXpX3FkR2APlGXhD54euEpPWHbFZKrxoWdFzeqog'}

# II) Authentification

In [ ]:
# pour colab :
auth.authenticate_user()
# pour accéder aux fichiers google (google sheet)
creds, _ = default()  # or use service account
gc = gspread.authorize(creds)

# montage du drive :
drive.mount('/content/drive')

# III) Configuration

In [ ]:
group=13
direction, starthour = group2direction_and_starthour[group]
image_dir = f'{group}_{direction}_{starthour}'
tar_and_zipped_image_dir = image_dir + '.tar.gz'
starting_image_idx = 3

# IV) Récupération des images :

In [8]:
import shutil
import tarfile
from pathlib import Path

# 2. Define the source path (change SHARED_PATH accordingly)
shared_path = '/content/drive/MyDrive/' + tar_and_zipped_image_dir
dest_path = Path.home() / tar_and_zipped_image_dir

# Copy the file to your home directory
shutil.copy(shared_path, dest_path)

# 3. Extract and delete the archive
with tarfile.open(dest_path, 'r:gz') as tar:
    tar.extractall(path='.')  # extract to current working directory

# Remove the tar.gz file
dest_path.unlink()

# V) Récupération d'une partie des labels :

In [9]:
# Open by ID or URL
sheet_url = group2sheets[group]
sh = gc.open_by_url(sheet_url)
worksheet = sh.sheet1

""


# VI) Annotation

In [ ]:
# get dict_labels
dict_labels = build_initial_dict_from_images(image_dir)

# Complete it with data from google worksheet
complete_dict_from_worksheet(worksheet, dict_labels)

# get all subfolders :
# Root folder
root_folder = image_dir
# Collect subfolders
subfolders = sorted([
    os.path.join(root_folder, d) for d in os.listdir(root_folder)
    if os.path.isdir(os.path.join(root_folder, d))
])

# define button color dict :
color_map = assign_colors(dict_labels)

# Inject once (put this near the beginning of your notebook)
display(widgets.HTML("""
<style>
.widelabel input {
    width: 100% !important;
    font-size: 16px;
    padding: 6px;
    display: block;
    margin: 0 auto;
}
</style>
"""))

# Extract timestamp from folder names (just first token)
timestamps = [os.path.basename(f).split('_')[0] for f in subfolders]

# Assign a unique color to each timestamp (simple hash function)
# color_map = {}
# for ts in set(timestamps):
#     random.seed(ts)
#     color_map[ts] = f"#{random.randint(50, 255):02x}{random.randint(50, 255):02x}{random.randint(50, 255):02x}"

# State
current_subfolder_index = None
current_image_index = starting_image_idx

# Widgets
button_box = widgets.VBox()
image_html = widgets.HTML()
meta_label = widgets.Label()
meta_label2 = widgets.Label()
meta_label3 = widgets.Label()
status_label = widgets.Label()
result_output = widgets.Output()


label_input = widgets.Text(
    placeholder='Write a label & press Enter',
    layout=widgets.Layout(height='40px', display='none')  # no width here!
)
label_input.add_class("widelabel")


# label_input.add_class("label-textbox")
label_input.add_class("large-placeholder")
# Container for image view
viewer_box = widgets.VBox()
layout_box = widgets.VBox()  # master view that swaps button list and image viewer

# Close button (top-right)
close_button = widgets.Button(
    description='Fermeture pop-up',
    layout=widgets.Layout(width='20%', height='30px', align_self='flex-end'),
    style={
        'button_color': '#444444',
        'font_weight': 'bold',
        'text_color': 'white'
    }
)


# === BUTTON LIST INITIALIZATION ===
folder_buttons = []

for idx, folder in enumerate(subfolders):
    folder_name = os.path.basename(folder)
    ts_raw = folder_name.split('_')[0]

    # Format the timestamp
    try:
        dt = datetime.strptime(ts_raw, "%Y%m%d%H%M")
        readable_dt = dt.strftime("%d/%m/%Y %H:%M")
    except Exception:
        readable_dt = "??/??/???? ??:??"

    # Get third image filename
    try:
        image_files = sorted([
            f for f in os.listdir(folder)
            # if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        third_image_name = image_files[starting_image_idx] if len(image_files) > starting_image_idx else None
    except Exception:
        third_image_name = None

    # Get interpreted label
    # if third_image_name:
    # try:
    label_text = get_interpreted_labels(dict_labels, third_image_name, correspondance)
    # except Exception as e:
    #     label_text = f"Error: {e}"
    # else:
    #     label_text = "(No third image)"

    # Build display label
    full_label = f"{readable_dt}\n{label_text}"

    # Create button
    btn = widgets.Button(
        description=full_label,
        layout=widgets.Layout(width='60%', height='23px'),
        style={
            'button_color': color_map.get(ts_raw, '#dddddd'),
            'font_weight': 'bold',
            'text_color': 'black'
        }
    )
    btn._idx = idx  # attach index for handler

    def on_click(b):
        show_image_view(b._idx)

    btn.on_click(on_click)
    folder_buttons.append(btn)


# button_box.children = [widgets.HBox(folder_buttons[i:i+5]) for i in range(0, len(folder_buttons), 5)]
button_box.children = [widgets.VBox(folder_buttons)]

# scrollable_vbox = widgets.VBox(folder_buttons)

# scrollable_container = widgets.Box(
#     [scrollable_vbox],
#     layout=widgets.Layout(
#         max_height='500px',      # set vertical limit
#         overflow='auto',         # enable scrolling
#         border='1px solid lightgray',
#         padding='5px',
#         width='100%'             # optional: full width
#     )
# )

# button_box.children = [scrollable_container]

# === VIEWER BOX LAYOUT ===
label_input.on_submit(on_label_submit)
close_button.on_click(return_to_buttons)
viewer_box.children = [meta_label, meta_label2, meta_label3,  image_html, status_label, label_input, result_output, widgets.HBox([close_button])]

# === CALLBACK REGISTRATION ===
from google.colab import output
output.register_callback('notebook.on_left', on_left)
output.register_callback('notebook.on_right', on_right)
output.register_callback('notebook.on_up', on_up)
output.register_callback('notebook.on_down', on_down)

# === DISPLAY INITIAL VIEW ===
layout_box.children = [button_box]
display(layout_box)


In [18]:
# Vérification sur le fichier google :
print(sheet_url)